In [1]:
using DrWatson
@quickactivate "BNP2"
using Random, ProgressMeter, WeightsAndBiasLogger, MLDataUtils, BSON, Flux, Zygote
using MLToolkit.Neural, MLToolkit.DistributionsX
using Flux: Optimise
using Revise, BNP2

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/kai/.julia/packages/CUDAdrv/b1mvw/src/CUDAdrv.jl:67
┌ Info: Precompiling BNP2 [11504357-4fe5-5405-981c-8cd43ea31635]
└ @ Base loading.jl:1273


In [2]:
args = (
    dt          = 1f-1,
    n_trajs     = 10,
    is_noisyobs = false,
    σ_obs       = 1f-1,
)

;

In [3]:
function sim_traj(ms, qs, ps, dt, T)
    env = Space(Particle.(ms, qs, ps))
    traj = simulate(env, DiffEqSimulator(dt), T)
    return [env, traj...]
end

data_raw = 
let ms = fill(5e10, 3),
    qs = [[-1,  0],[ 1,  0], [ 0, √3]],
    ps = [[cos(π/3), -sin(π/3)], [cos(π/3),  sin(π/3)], [cos(π/1),  sin(π/1)]],
    T = 50, n_directions = 10, n_moving = 10, n_speed = 10, n_initials = 10, n_tests = div(3, 2)
    
    trajs, trajs_test = [], []
    # Trajectories with different initial conditions
    for i in 1:n_initials+n_tests
        traj = sim_traj(ms, add_gaussiannoise(qs, 1f-2), add_gaussiannoise(ps, 1f-2), args.dt, T)
        push!(i <= n_initials ? trajs : trajs_test, traj)
    end
    # Trajectories with different initial directions
    for i in 1:n_directions+n_tests
        traj = sim_traj(ms, qs, ps .* (1 + rand()), args.dt, T)
        push!(i <= n_directions ? trajs : trajs_test, traj)
    end
    # Trajectories with different speed
    for i in 1:n_speed+n_tests
        traj = sim_traj(ms, qs, rotate.(rand() * π, ps), args.dt, T)
        push!(i <= n_speed ? trajs : trajs_test, traj)
    end
    # Trajectories with different moving directions
    for i in 1:n_moving+n_tests
        traj = sim_traj(ms, qs, ps .+ [randn(2)], args.dt, T)
        push!(i <= n_moving ? trajs : trajs_test, traj)
    end
    (trajs=trajs, trajs_test=trajs_test)
end

for traj in data_raw.trajs_test[1:end]
    HTML(animof(traj).to_html5_video()) |> display
end

;

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAA1Z21kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAACSFliIQAN//+9vD+BTY7mNCX\nEc3onTMfvxW4ujQ3vc4AAAMAAAMADuPlocG+vHIc0AAAWwAGWbfzhEj8xQEA3Pb/v95jiX87sMS2\nZoJWY9un2QhQjDyyuk7jc4khdvroEeZyk0B/X6NKr+aDMgHQGoVPquzgaVt7L0V9rJM+oF3oJ7A6\n7FJHcJyhNlDSjlK9080RT63aQRhV4eYNe4qxr79ABrXAvKKMnYAR4ZcALlXg8QKxwVBUE1FiNHTY\nInzeKjjlDWbqDI0ddN9MFr6uQWaf4+MYyF5Xjuk7qJozF0TFtsBRwz2foigsmy5CO57aU14vMoKz\n54Ol3luXs/kKAf8KXCulff2PrVu7Az2zXaXe6IMfnK00dklszggTTU8+QNxBUGHwsSFqTTuaodf8\nzS/Zxd87GRasgf94T3duftk/2i51jxStoh67qgHuAPxHPi/Yoy4/7mZPrf6v/hiCxiB+2JwVuJqT\n0MNnPP9V6BetWSUC+PtBB5s3Ab8zx6xTp46tSYNO+I/t9F5Qruyc/FoDLB6b1HmOpUjEDvNKzJNM\ntgvaXrDZ15pAdkxmKsmFcaHBPMjDt+9zTDGL4jHA0utkOS/xcCuzIz1JY/g0/QqpWCBFcUyw02Ed\nA9PofGMxXgn7wbbyfcXrYlctlhi0T2iS48kYM02wA1xyEKePrX2v/Ozz7d/hRF+ip5wqExK0OEbS\n9S9C0EH6MJkGm/jSxJAmJGv8smeAmW+PuR8Hd1/ABjP/pnri+As2iPyaC9P/elTWA3T2jdCfTbLY\nttO74m15buZHbLhHyQoR7d3zXE/Q7x817GwJXY7/wRjXccGQ/cp2EmCNXQ5VRidD69OSFQa9i7lo\nhhNpn2wvWOKqDIWEABWRu4dM9sGoJGDWjqhQN7oQs8P/7HPBlgkjjAyT4YpZJBF/gX/nH2N8DVwq\nQ7waoGZdGg6g3zEvYRYAmVYQcGbgiGqFHm+kYbLjj35R5HACDA3UpnnsoQXLE39jbT5pRRK8in2W\nve1eexQLyJmOzIDjhdYdkCmSQfhJtb2+x3j+OIA42eTIU7pIF5KgSDRXuUxZPzj8NCXSOJZ278C7\nwiFwG8wBXHASQxGXaWLXypd0CDbwRBd3UXbwHQEBxH+BQpZ3Xuo/oO8hLz/dH9rpYzDFZMtrIuWu\n+j2k4p2RHoIcIgAxwAa6vzls23qg/y/SK3ZQOEzfTxWJs53SF7thWIZAK3iJGDTB0EywhGuPPeuM\nueMX3CG0ark9UsWOYKaLVmIwM64K3IrdDzpbwHkgBQaP06L/S2ThxoMtjtpnuth8BuK6bHN3uqI/\nI28y6dNo7FQKXAaZpSt10mMOmedel8lDqYUN42zNKI89i2Yj4uhOoiTbGdC/pv7tVxmHxNYJR59h\nmDROa+xZTnwfJk1asPHLtdak2swQXF4xIZNydi/4BbRPKNNZfyMVGC9zhCDQmwcYyRc/eMvbffj4\ns69EbT64Iyc1atoMNitiKiw11UZvkeiVCyhSGb/sR59jZouSlyIJXis64bYGimIHOj9NYA81dAOQ\nFardB9kr6h24nChyd5zUIytDvzy3h8tWD/bCniQqh5EyiA95aryoKicn4lGb2668zCrjVpdMgm4+\nrR0zQfPB37dCSdwXQ1DmPBUhqVgYKE39XX5uNZI1JaXXaxYZ0VK4AK3zB5HiknvouZfpVBXzK28T\nRkZAlpCDmCrDNvTGJ1wpxF0c/zFPfUPWVynSOq4vd+KIAzpATZ6dqofnLQrv4Kgp10N/UgPX9bUU\nM0Ob/agvBY04By/fhpQRWNIspyTbZVneV7hLsKWhjcdtLlbBUeaq+gRHbZrfyJhKvtt9N50i2bX5\nWodcFRsbnp1/6kLCPXEy1B9Ga0LI4O2smmH/aUacXqVcCd73KYP+F71RLPPuEjW7PdwlqDyyEDLF\nv2i948b/SxTUy4LYBD9ipWVdDwWEpT2XffnjGyAbg/RnodJdQ9w5aL2n5gMSVS/wgGF7Xbgxj7qw\nAiU2/016DFiOckI/F4xFh/umSJdrByfGmCG4UwE1vDzA2+ttHDUmd15TDPWSE8hrLiPhg2OXReR3\nH2JGimJywhng5ydwc8BgymF8g1mufup8t34cDSBWF/IFYS1GAAZQqj2+aKWug8Wyp/fcj0jCSakK\nrb0dRzQOvUc09q6z4psN8C809hnVBeMODRrHfMqIgX/ZLPi4bsxLh4npLZWmkXiBJ9CQLngQa1en\nz0wem8xt6qAIIFUW+ctutjKO4k2qpmOLxUq3KHyhlSNi5/I5tyifxXkZtW9zUUh///tvEXTQsoX7\nkKkEs7ycAGIvs3cE2/QtF8EAIINKpU8+XaZBbbxQoBHP+7QdH30UBVUkqKL7mFyvfl2UPqASWLeF\n674Q/MLvgIpx3sd/sNvHvwdMBMi4rdrTaCY4cMQKn81cA6iFj955mhT0rPqwU2ovuWrSFVgL5Dcf\nRWeFlAbRcoC3q7BPKQj34mHiSEEAo9D++pJZcTXmuiSDJKKfuy98LX/f7p/wbakHeZZFhKjbMbvF\n059MMXCKlvMY0Hr49xhrBPRFdVIdwqINfgW+tlYsrx2WDbecSJHeEywpAKZ6E6efy2JhxqW0hbsM\n1GqoiKDmRDMqpzDpVFpQfD1V+p4UfEdBgMubN60AlmcS6WSwy0UNqTZ2STDoApskavOJjM8w3qcU\nOR9zP9OykhMz/vxph2YX0zrfM6CrppL4e6subzJCtvr2OJm5z+JaA1jRa1f94oAYhQEW2Kkq3fax\nUuqJF6r78UDXN7PM1pOgfQhhxtlxLbKgVD64EiYVoh0EFS

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAAzC21kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAADyVliIQAM//+9uy+BTX9n9CX\nESzF2kpwPiqkgIB3NMAAAAMAAAMAJOccxEpfbYdmAAADAUQAGoa/yxEj8yqnfVoNO7gfV+TIv5Tv\ns0iHfIr1p7IP9g9IAvB9q1rYRySRumMDS9+YqUAhe7lyF8X8AT79oDzmeC6Knnx9xACQOZzmR8tp\nKis21BvfmKvMLR75b/KxlI+O7kkwMppPqv3/qR1zVQMExxuLlR31sswOKFiZel1dg9KoUBRWXw0C\nXYKikoFXB6wMU25qxdrnxQJd0wGAPrihUtBwzdjz6oV5CdJhP3xMDN5w8e/7napI8FGETJ39IIw/\nyB91S0S0qtm+5DEj6Ymxq2lo15qdJjqL9a9Qp0h/HHtYlNdMOWh6WPbGVPbTXNwQcO5BAiR3Kz54\nOlzse+jJj9wuM/rUOkpP/fmUkA8mX7QbDZGJSuSxzliJBqKNR2h6R5ugG+ABj4Afgr9eLXSeRQtF\ng61Mb5Qf4GBUCAkxXb1QScSR3OMN2S39JtNHWPCgTs+B8G5Op4yHPZiiuF4/IcWXeYObQoIu7ePm\nwC/SbsQn+vNfRAChgLekEMBu6RzS/t9bE6NKnUa5/GUhQSd5qJ5sNueZvDDAH7syjQazShQmvn6j\nHnGLUBSD00j65NIfHzNsmbUjr+1lq9AQAB33+w4yA5oMajpusMnDWNpbdJNRX91Z5Ggf9ng+bUi0\nRAt2qvzt63z6X7TsgCKXypUHCDYzMBuM6I4l2sU75HXWQj5lA4Sxo/EDXYXzWW+u86pHXCdEZSuS\nu7KbNbAC2iI/DBU+n1CfjJfDNrwlFIO0YsZSwCx9+ovZWm4o9YcVckACz5npDytC9i/f1OcE8KSF\n+Vna+9ZIW8EA5UajFVkot2YeIAUT502pWDTvtZCwNFnsdfkSG0IggXzaxkOSpldWyM8BmqrR0GBQ\nVrKK2vZG/f5KwVrxyodwDLbfBJ/H+o2b5f+eqiBPnhK+rzML1l2ncbLd+4BiU5H/r4Smck0nBA55\nf/3DcsmO8GnCJexKod2z8JfxTiTOIxUtr93qHP0bwipO2xwnnhtppmqiZ73SSaDNTH2XA80UytYF\ng+PACmi2s6zTpOBzS0/wxg3uxdtmsQsRqfU8MEGxrMuHUnveTdgsfyTY5ztT3fcAxz+Q8JDP6HdG\nj4wgx9Fp2P86LBAwhBNcGKBie7/zTiYZ/tH8qHa4dcAOi5BQBTHm2rcQpqVBem5er67l7RcHrhr4\nygLi7qUjKnikyZJEfqcFo2KfkwU5AGOXJItfRqnGV7+7hXts4q4N7YEbc3BU6puk26S/6Qlncmo7\nFyUuvligxk23cr8Hf6syhdyCq5gYNJi9Y1WOata1rwrR0FJPFrdKWCasBcj1p4E1kAOouTthMfyP\ncmK8pIVKJbN5dg5nJVH77iMANOm8/wjOtTpiSRdQV3JU5KhTfRtGNGXkfSlTckJdU/29VH7KrzvY\noSXlT59Z3Zil2DARTMgCTcYuMIP1utSx4RE02+bNn7mxENc7m7v82jvBtj06cLcu5bpPplcOcIix\nNDSLm34dnwjrxOcUwRRVnevDyp57WEczRtvWvcx1TyoXLYafuvldGGrMkBXhD/73VCXX79dkNHr9\n0DxwGGptmqS6/NIwOP+/y0/FavgvAi4nSL8p3Kqp2QcO3mkShXhJYhFmyVOm0gmCYGz7sq3nLCDf\npccW9+/k2rv00xlVWaT0cdGzKuxtyzZWElU/cGXcF/0a00WyP/12XX2ItGWavcTVZ6A3bwxqGr3u\nhzxWeIt9sQPnzMyTlFmqXWUfo6bbMQDPqQNxe14riUb2xIZpIRmqXQ1nM+dTnCViHMY/s4Pd81T5\nrDpV+7k9TCq/ir2fIvlmFZN+ajgW9pLLRkndsPRfVWWZ3FjtzFmFzvq/tUJQoA8/0mnQZXNxdxSj\n0Mom2cKyKuQAeKCT4CvPuMrM/3lZnqHphdJf/OqkggnF0yg0dHTGInKEN8L3uipCLGmYR4VvEVBL\nhxvYxuyk3MfPm7Y14+qR6wOq1d3En0ibAEJ10oz1/7ccFFgCblgc+EMkO7lS18gkQJUFCE3kRRPT\nNjulZpCqxuTuTG/kxpwmmjYjWOZDeUwAJVOK9Qgnvn6Nbc7RvuVCJLQ1Fkjp5cor5PcNffj0543t\nqE82Xc2tQb4z8zRhvtZ+mzhdKMykjtyRkuQIZuXGA4JlSudLbfm2YCzPiVawyBDHGQRBUjz2oMQj\nPCz2IGF4rxw0UjIq0tDsuMG61DyqYsTTqCLi5gTTCHg4aFqF2AmuBfhvYFvijLnmglPXoC0PpctM\nTUY9pkHzbAbiDgt3LW0NunTyPVexorAtPjd5pJ6WqWgo7i2OcMBn9nrcKuR5gNlHJGBOM+hjZW90\nLZXwFQCRQ9gpaCIGtmnK/GGlO6iiAlhVNQzCYHyWNatwAbQDEAMPtP1IHlL6WruwVMVlHviWtWYb\ntUqwk4Ju8ytVH2ilNWBQZir0c89hQvx40Rr/ocmU9b/L9R3f8u1FfPxr8d5XktwKrzIc1bmFufVy\nyvmvvJJL6s3ULW012T2G89rTPeJvpOOacHz6SgOd+aJ/sxB3TjUEkKHhQnpFiHGxJ0lzM0uUoqEY\ns2JPxkumEjPgvJBUEDRhh6cnw3waXne4RReL2V0Ga0sAjC4avVHzLlmfTpf43fEnKfEa9RRPaGm8\nTlYCLgQViLUlskhIS+5gCQRPkGdhh/KUi++F3Ewx5Btrt5zqzO1TbCM4yzuKD5KOGbi4oF+4ywW0\no23bCDaYxjWL6cv8DZkPPn6+Hw/mLFs9jkdWsufo783JfY

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAA9xm1kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAACpRliIQAN//+9vD+BTY7mNCX\nEc3onTMfvxW4ujQ3vc4AAAMAAAMADuPlocG+vHIc0AAAWwAGWbfzhEj8xQEA3Pb/v95Fxto07V/I\nWQOh26ujrV8O6+MugcEpCsPPQPNSRO17ZaKxBe1QvQ9ODo0qwaAMv0MGg7yK7acoO59SQ63DyFLy\n7R0lYFNyAzWS+P0WXuuBY0uTlLHZDBqRzxTupbGelEU/IzySty7yiWMA6bIG+igqjfgp0O45nXMc\nojPppo/YZqHCBqJTGSZo69ebsvW3gd8wLVqydhT15wKTAv4AAAMANkULUcv2JFsQOc8HieWNmnYu\nB5VsilhQ//LslJd4Sd0ZMrENE6vzThcqJXti2S9OYgNdnFTD9BVK8x8vpHwJ3X4rHL1ST3/ovE6i\nb2EMn028pDLHqKsEBF3qiyp3Ky+HO0hLI08qnIOMhjxKzXOMylSgPTLhD6yGhI3ov9Qit/g754vE\nBGqtwYvcwQ/3kMMJ4KKGVcATyvKBD+aI+DQAZ2WiSm0Ey4h0s+2rTZB+QAAAAwAO/MgtKqXu6ttL\nk933Or/AFoZ9c2JRk8isbysru1vqovA/vAXmaDF6Cg7k1J99my6SkpNm15JDXUcvrxiH6Q9KhWB5\n5wC6YiG0LaEyGFY/0uuWwXrV0LetbU4RSn9NqZanmpBNdO6bU3fgKUNGpKjuaKtQKuJu9DStzpKC\nPboUvpbVqKI/+wI+3wQ6MIgjHEmRk840P5qO5a20E7NePK3hXV8HyCCyi6rew2l733D/Zp5/QKRd\nfco6OncuTF+wxevjOQG/U3CakxjR/DP/0I0f/t+1fa4cWdXAO/IS645JCD/i4kVC6WUQXafawAAA\nAwKuXfVlSlQv7AAALTGt7sz2e5aYfRw3W1//41X7vLOdyaqDTuRwHKvBjI1xYSgVlHMOZEoN+Lv1\nWxORi19WBO62DvoChU2yLoVxhlYxmJKGnjfbFKFK/hp4fCSMtcYmZ3PQZ3c9wyy9sjhtWfabKYPj\nSS3IOzO4YVua5kiA0ByJDxUl1o1qdT9wYwjj/zibPNr22DupIHNNv9b9bv5Xx0Awz4iR61rnrzcn\nyAk4xOqGqjBjpqz+oRqoaz60GPs8O6ae7WNHbcF/0atKhVxnyrSl3470qXdEaopbMbBYQa9YNv+/\n/AVr17Z/66G4IKJaRSujahW0P4PWV//wqolHWsQ2BF7nFYFNQxQtYm3tTdFUAvO1+CACnPWo82pC\nSYxM4pl3wJXJ1MIAAAMAa+N50H9A1gBy3wRWLbvpURJKxfMA+z9pD3YF67zeqN3XzljxjnWBrMFB\nNSEXwmRQcNZV64fwCLeiHglK74YjC6IEh9YeUXqNbn04ZAVfUlWLAdCwLc9bXz9r6XdJqk96VWZv\nRBpyu//geqAd330MExqni1Ye/h8/j0gTvJhXCVjBiz+t4nV19OHjEIfe2QTD/QU/68OITdpb/frW\nUeLCmY1IxZ/ifojtMRVYGkk+gmucclzemaCZb3zi85UMFMBPE5lxgcLwlNBY9Y+NgvsGgTSAderg\nEZ6zRRzQ8NmqOXavxmQ3BXy325i+rBGqW1XjXJjAMncKkXwk7D87yWFfoLXDxjBgd+yUq7uASZCn\nSk3X5EQNO78r3b7Uz9jV/kJSp+TcRDXbZFeDMMxfbo11/anYqkID2ib+JkPyY5yC3KFFZY3WRzcb\nxOZRLmzTGP2o8WfnjT4WDKmZoHgTK51YAEtQ/PDbm04or58UPZpQ5fGIPdayVY/1XazHwDTzLx5L\n/K+N0Z5QG6zTklnTvrkNcQkoZ2elbf5ORhkbe3yXEJXXUOp5XJxrrxrKXobsoK6B9Uw9hm2p1J9m\nNsLkAAADA+AKUqvHy/ALO/gWp673hzJU6K75e6ykqF/KZ9WafzHfh1bi8UbygwQBVI9q0KgATFkw\nC9IC6ciE50b4A6RPE7nqgB5Hg2u4502A4rLwqVay8TFXuxWd9Zi3dD8452M8NnD2ccYwkIdOgI1G\nKjJnJ5m5F+q2NPrqh41iPs5vdgL6QMOgXjLty1qZDskdSxXfx5Arhw2IPIQ13Lpm+e9QAZJW2RLR\nQOmldt8xDrZxLcoQ3qsAJQ+Mn4ytm+GA3UB+0oZVWj0C1e1Mlhla1rn8KZ9qSCGx5aDD6FM8pi2C\npcWyfQMS0qKMV+uQqCrQDx0wS6RyngQMVLcupgAAAwA+CksoSJJowB6k9aceXkEgclUsRDlDkMHo\n0f9ulmhCaWZeqEdw4IX680EjMn88gUejzrT4wWM7kKZxWky7tgHbUknEuM2r4SgB9ro/3GAHfaRq\nYbkhTmLXl83MafGeBViMBZKd/xwdzwxg0FBVvmL+H/Khx7NWAzGZRb68rr3i2KLQWo+PkUKdzwm+\nNpte64kwHBY74Paj7DJJOKgi1+7rTyF8chjSFox38/kSRu9hMaT/JS/tt0hP3bEZhjeV0PxKNfVS\n1b+NjMZhQa9D4hsR6RMQlRDDJOz7yeuDGI1CW+esxyRne4ws19mTro41ZwAAAwAeUBbTuw6XAoVl\nPgc9mhegeEfTl8s9axZnVG2oR/6C+QB+dqfMl6/FycwUhn5VvljO82r0GKdMzg8z048fxc6PtQTM\n6rmUrx1tmpd4fMflHIsXg2xj+GdBxSEaZy4lp0E8UNnQsKYSIX41dZpsdt7FyinHxCY94T7koomo\n82wXOhKPbDSLgxweEjC4BQ5n8hrIGgdcJTnwD79+CwpxJ3fFDZ7O7d1BIAh9uBrCkbeFR/FmiVDm\nRfoEeNnR/Ls0GCzwZlS3qtOZ2C5UIUb5lD1Z7JqFlYDWGe

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAACyrW1kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAACNpliIQAN//+9vD+BTY7mNCX\nEc3onTMfvxW4ujQ3vc4AAAMAAAMADuPlocG+vHIc0AAAWwAGWbfzhEj8wHs1/l2ycsrd5ktf90AI\nhGhGmtkc3FnHFj6P51SVa5dlANiy0p96Tc68v+omsJJR2dkD5FtPQU7jqZqxenmGCj+Gq1KuJ9iS\ngSP/XOpRquoSY22YsPsejotTg4kFZxM4OuYoqmW4lXOGY7oIxo5dj56Tk9MWXvysB41r0xHRz7+A\nwHXa4tz6zPlDHaj1Ofecee6kc7k6Qv5ZAATnNNxLcbBV50Xnh5ZpwZoYTEMb3cwg3J19QUmMtvek\nM60w6SMn6lmRbVA2BMhPjF2va9d0fF1crYhf2ZOxVUdzryzIl/9TOb8vG4rkb9sqqNgvApP5WPQC\nSAlKR0StDc5jAPnbdMRnbgP/1itRYNoB0lk2xoJvc29Tg66kp4ayne4SCX6mkpHZGT1POI9ptFky\np8w13MALeIABeLeS75oZUTMw/CT/c8QcbGRLUFOwoX9RK3WCcLX3GpBBB0pw4ZU9/yGu7kAj14Qv\n/8RVE2eHuX5dfCi3f0OYzGdLDPKFRlSeUZrjLpVjNWfe+Ttc4crKedeEWxKR98YeL+8PpXsl7XYR\n8WU53LgV2H5PYbvE18813IUbhi9X0lPex/GuuCbID4LCTZMUrRFMMmemp4xYeBIcG0iVf4Kk3Isr\n4h4B2/SvCdWXcmIK8OzVbW0DPDhnkmSHnHXg4u9K9Ql16tIlacfTJxkYhnVIM4oGZ77X9ColRDhv\nBUeF9L3BvCYJpI8Jkn3tQ2WC05rZFLuhzrp/yv6E6DujuYSeUJy1YxC/C3hxHIXSYEua+KpNRQdT\nnNVXOsvXGfFi2BTDyNH0AAXbFfplIMMRCblTzyAWe73TNU6CcvobJw26an/1r9XhBufAh2InPR52\n4Z9QPHlKD6FXYBqn6zLNVmY/Ad3SNO+AYXqm+G9sp+ZysDNH0E6f7fulr+78oS3tFW1dOhcs6qR8\nW/FuIkSjyfVRTyRG8YH+vkkxIiBI+Y+UDSndVqW6B6KUQtD4P2Q2YbCQ6707lImEBwLGluBavZxr\nWBJF+O6ahXDl1xUi3poiYAm7gZ7NFxGNrcgpz1mr2YeyZlcA9UvC19uRevnDJhPHuxTwsAd/4R7X\nkoketOpOswOxsu87bFssmJGwvB8UZk/Kn//0it+yigOTBYFKYBssmhOcSVDtKZ/PujE4pr41qqL4\ntRq6W5bBwzqIhUHd3x1wfOc5sCYnGg5wyUx31XsTTvtPvLql7uuZahTwtB4W5gJzfAO3w0/tv43F\nI1Wz3NO/x5BdXQLlUgaFwUxLgPWxYJwPmUX5WUqbBzeiqcvuf6MhzhqDlZCcqxGzfxk4kutYtViZ\n92TIHumIDftxGBPECVC0jhiV+1RCJ9XEP9UO47dNybKS8uX5KdQVOjq9p19jFzwmVB39rb6tQiVb\n1r+lvh3/XkNWi48+jlckA/Rfm9h8GNbzEmH53yJNP1PGcXRpcJ6YM2SiAy2W9dc0ORB8gZ8gq/sg\nd1mdQDAPKHpMcxoCVRUEDTEiShmBbu/pLyC72P6YoBSn40fTwHsfVlJjAdTx5S+6gAAFDxn3mHH0\nsXEcDUeSCEE37xxjyAUHp72DPuX/4yxD0DyAJnrqOLDa42CV+fMslUA0/rzfIDSK0Kl0e9pzbs2N\nj2+uiUHmV1KniENhwy1lPCPc0NSHAvk5mSaZQ1nl5KSRwGC4vcCw2hFnD5ZypFDg4XU31qQH5dSS\nU6gz8u/SIEj97y90CkKCTl9diNCtFPsKvn/fxY3yLGEla4T7dlnT4F7QGd339KlS5QRiM79dEgdJ\nVbtcpCshpgdONdsrHkfHfNpVjm1WQ1lmr95aSWx0deBZPg39E8tAVZANfo5RdzN4D1Snykf5MhEm\nAToPjpPN9rxCagSX2CAJ2KMcABQhyKxjhxzTwM12FUpRQ+F5w58VZT5pLFJ7W3mDJDYABBNAenx+\n8hK0pqoSafwIIEHCzhFE2pMUrbZOlEwNkvIWkNUBE2BDYwSTawCP3GaQfIOVzjLBuawau1C5VWzz\nplK4Ym4QZp/aubl57A/b7DRdpk90J8wGedVnNfyg6CDTTdcfuPt9rDHMc7dBW/LgzQSkfPQ73Q3g\noze7fjMlg5J/rKtzBUTjbS4622Ysdemx/Y8W+1qfls28FiPRJaiH071XiopygV+q1ZV+5fKH3Uhp\n/hL4lXS9s2058vN5fqNQg85qNedMJeSZfZlk22zDkIkLOri7Q1y54acMqjdUiAKXYCrnrUCjO0/U\nfcJXg3AjIpXOoZErwPrd+MqyEzdcbtY+rddBlsUrnTDbOaY4b38A15ueb8rOtNE4KXuHgcVD59CW\n+YlhNEhQH1aX11JpWdhwmxVmD5m98wmylGACliVReiQZlVLHn5FLiY9GMuvQiD0WVzXbeN2grPVe\nBA3EUC56SCy0SSUtI31NL0fXYRS1dIYCPw40XvgfhWY3p4ypvxgA1UcIie9ttv2b8P5aQ+q/Aqyi\nfSiVzuznweRk8O65qI53j2rOQX8Zl8qUzFQO6EtP54NhiSqIgJBaAZ+0hmR+rADk58dUJFswTD82\nmMlNy3qwEjJHvmjmfmWJP458g4n8Nyy81VLeX53He6RCVvxcTqlhXF2HHQ5VI/2FWQ1oUvB6ISUy\ne08A2ZhMidwJ2mRxI6ZGiZT5e8WaylbsqO2f+oKjyd8Z+sRF6EsbIyGDFxC7ttOkS9dORr/u0FAm\nlScRkMUOoXl240PqWBfqPgfHfqGpqldRIpHKVDjH4PupBQ

Assume we have $n$ bodies in the $d$-dimensional space. 
Let $D=nd$ be the total dimensionality of position variables or velocity variables.

- `states` or `s` has of shape $2D \times T$
- `states[i]` consist `[pos1, pos2, pos3, vel1, vel2, vel3]` where `pos` or `vel` has a shape of $d$
- `q` and `p` are postions and velocities for all bodies respectively
  - `q` and `p` both has a shape of $D \times T$
  
Some definitions

- `attritbue[t] = [pos, vel, mass, others...] -> not first order if attribute[t] != attribute[t-1]`
- `state[t] = [pos, vel] -> first order Markovian`
- `visible_state[t] = [pos_sub, pos_vel] `

In [4]:
function Random.shuffle(data::NamedTuple{(:s, :s′), <:Any})
    idcs = shuffle(1:size(data.s, 2))
    return (s=data.s[:,idcs], s′=data.s′[:,idcs])
end

function preprocess(data; do_shuffle=true)
    # Convert trajectories to training pairs
    s_list, s′_list = [], []
    for traj in data.trajs
        states = stateof.(traj)  # Vec{Mat}
        states = vec.(states)    # Vec{Vec}
        states = hcat(states...) # Mat
        # Add Gaussian noise
        args.is_noisyobs && (states = states + args.σ_obs * randn(size(states)))
        # Create pairs of x_{t} -> x_{t+1}
        s, s′ = states[:,1:end-1], states[:,2:end]
        push!.((s_list, s′_list), (s, s′))
    end
    s, s′ = Matrix{Float32}(hcat(s_list...)), Matrix{Float32}(hcat(s′_list...))
    return (s=s, s′=s′)
end

data = preprocess(data_raw)

@info "Processed data" length(data_raw.trajs) length(data_raw.trajs_test) size(data.s) size(data.s′)

;

┌ Info: Processed data
│   length(data_raw.trajs) = 40
│   length(data_raw.trajs_test) = 4
│   size(data.s) = (12, 2000)
│   size(data.s′) = (12, 2000)
└ @ Main In[4]:25


In [5]:
function apply_kernels(X)
    return vcat(X, 1 ./ X, sin.(X), cos.(X))
end

function euclidsq(X::T) where {T<:AbstractMatrix}
    XiXj = transpose(X) * X
    x² = sum(X.^2; dims=1)
    return transpose(x²) .+ x² - 2XiXj
end

function pairwise_compute(X)
    dim = div(size(X, 1), 3)
    X = cat([X[(i-1)*dim+1:i*dim,:] for i in 1:3]...; dims=3)
    hs = map(1:size(X, 2)) do t
        Xt = X[:,t,:]
        Dt = euclidsq(Xt)
        ht = sum(Dt; dims=2)
    end
    return hcat(hs...)
end

;

12×5 Array{Float64,2}:
 2560.0          2560.0          …  2560.0          2560.0        
 1024.0          1024.0             1024.0          1024.0        
 2560.0          2560.0             2560.0          2560.0        
    0.000390625     0.000390625        0.000390625     0.000390625
    0.000976563     0.000976563        0.000976563     0.000976563
    0.000390625     0.000390625  …     0.000390625     0.000390625
    0.387587        0.387587           0.387587        0.387587   
   -0.158533       -0.158533          -0.158533       -0.158533   
    0.387587        0.387587           0.387587        0.387587   
   -0.921833       -0.921833          -0.921833       -0.921833   
    0.987354        0.987354     …     0.987354        0.987354   
   -0.921833       -0.921833          -0.921833       -0.921833   

In [32]:
l1of(x) = sum(abs.(x))
l2of(x) = sum(x.^2)

l1regof(m) = sum(l1of, params(m)) / nparams(m)
l2regof(m) = sum(l2of, params(m)) / nparams(m)

struct NeuralForce{T}
    f::T
end

Flux.@functor NeuralForce

function NeuralForce(Dh::Int; n_bodys=3, d=2)
    dim = n_bodys * d
    f = Chain(Dense(2dim, Dh, relu), Dense(Dh, Dh, relu), Dense(Dh, Dh, relu), Dense(Dh, Dh, relu), Dense(Dh, dim))
    return NeuralForce(f)
end

(nf::NeuralForce)(state) = nf.f(state)

# TODO: make this variational to account uncertainty in integration
function leapfrog(state, nf, dt)
    dim = div(size(state, 1), 2)
    q, p = state[1:dim,:], state[dim+1:end,:]
    p = p + dt / 2 * nf(vcat(q, p))
    q = q + dt * p
    p = p + dt / 2 * nf(vcat(q, p))
    return vcat(q, p)
end

function get_lossof(nf::NeuralForce; dt=args.dt, σ=args.σ_obs)
    function lossof(s, s′)
        ŝ = leapfrog(s, nf, dt)
        lps = logpdf(Normal(ŝ, fill(σ, size(ŝ))), s′) # elementwise log probabilities
        lp = mean(sum(lps; dims=1)) # sum over state (dim 1) and avgerage over time (dim 2)
        l2 = l2regof(nf)
        return (loss=-lp + l2, lp=lp, l2=l2)
    end
    return lossof
end

nf = NeuralForce(50)

lossof = get_lossof(nf)
Zygote.refresh()

let s = data.s[:,1:5], s′ = data.s′[:,1:5]    
    lossof(s, s′) |> display
#     @code_warntype lossof(s, s′)
end

;

(loss = -13.601502f0, lp = 13.622411f0, l2 = 0.020908466f0)

In [33]:
logger = WBLogger(project=projectname(), notes="")
config!(logger, args)


wandb: Waiting for W&B process to finish, PID 84473
wandb: Program ended successfully.
wandb: Run summary:
wandb:     train/lp 16.530418395996094
wandb:     _runtime 36750.425022125244
wandb:        _step 59999
wandb:   _timestamp 1583586713.00773
wandb:   train/loss -16.530418395996094
wandb:     train/l2 0.07922817021608353
wandb: Syncing files in wandb/run-20200307_031223-mgd3xaon:
wandb:   upstream_diff_0d40ca2888a766d1596ba1bd00716363712faa70.patch
wandb: plus 8 W&B file(s) and 0 media file(s)
wandb:                                                                                
wandb: Synced twilight-wildflower-64: https://app.wandb.ai/xukai92/BNP2/runs/mgd3xaon
wandb: Tracking run with wandb version 0.8.27
wandb: Wandb version 0.8.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Run data is saved locally in wandb/run-20200307_131351-45x8lyev
wandb: Syncing run trim-moon-65
wandb: ⭐️ View project at https://app.wandb.ai/xukai92/BNP2
wandb: 🚀

In [39]:
let n_epochs = 1_000, opt = Optimise.ADAM(2f-4), ps = params(nf), batch_size=100
    with(logger) do
        @showprogress for epoch in 1:n_epochs
            s_shuffled, s′_shuffled = shuffle(data)
            for (iter, (s, s′)) in enumerate( eachbatch((s_shuffled, s′_shuffled); size=batch_size))
                local res
                gs = gradient(ps) do
                    res = lossof(s, s′)
                    res.loss
                end
                @info "train" res...
                Optimise.update!(opt, ps, gs)
            end
        end
    end
end

;

Progress: 100%|█████████████████████████████████████████| Time: 0:01:32


In [44]:
let traj_true = data_raw.trajs_test[3], state = vec(stateof(first(traj_true))), T = 50
    function envfrom(state)
        dim = div(size(state, 1), 2)
        return Space(Particle.(fill(5e10, 3), state[1:dim], state[dim+1:end]))
    end
    traj = [envfrom(state)]
    for t in 1:T+20
        state = leapfrog(state, nf, args.dt)
        push!(traj, envfrom(state))
    end
    HTML(animof(traj).to_html5_video()) |> display
    traj_combined = [Space([objectsof(traj[t])..., objectsof(traj_true[t])...]) for t in 1:T+1]
    HTML(animof(traj_combined; refs=[4, 5, 6]).to_html5_video()) |> display
end

;

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAAtTm1kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAAEEFliIQAP//+92ifAptaQ3qA\n5JXFJdtPgf+rZ3B8j+kDAAADAAADACQC/Me/oQdDNCAAAE9ABF6c/uDwP4ApBwSzWnuXl6LmlbQ8\nDB73f2CjN4cnKY7usyIdinFGP3WQ0/H7BHMwaM060lSbHSM1kVWvjUf31B0gcYGBoXQiP2j12o/0\nT6Ks5/hKOrCjpMqmwXcNdsyGxb71tuA0FujKJ1CclGhYjLjJsTQ3rznxY5+sxaCtAhPekvJFeazW\nqXYmvkLZ7rhKp/Jb+JUEIsNdP6q/UBmlG3FRo2oPvqnR4F/CgpoHtEvjuqZCXqrug4KYcAAQxQXw\nAjTnytCFcE+3a8M9jAjqkHAJ+o6AupmYUFq9kUFxZHfGCnamaYPf3PzQ+ylS5OcrzbBiJmbYhJYr\njE5RLC7BOcg1PCWjIijtNM0d6CwLWqPDGlaYdSIv6S+zf0y9P3tEyY6QEZljr7ocBt2NGNmnPy34\n2lo6h9y2GEFgbeG53SNsRYSSkVG9UvDyBTf9Q+I9i1Y644cxX/GUMYwOgv9is+IykAAzCAojsE+e\nHsFlyFz3iC8C/pbU2rDw/p1jdgRpZkg+oKk/f78WEJ4CF/76ctYAumygfkMZ0B3Y0fxRlw7770aw\nq3AR0EuKyBL6q14CPaYobLHZNn3usJ/MiGuZZMiTArd7JK3CtLRLXN3/GICi6XSPnDGpu7sQWwvD\nAwAMOwXShCUiPz1shQ8kHniH2E3wJuEfNWD0ANQn8Lg/Hh99Wg1kAAAF0eGl/8/6Ds7phj6w+yUt\ndEHuR8amLf8+fWczJF5/Ew3BNMg191/gzsz5yl1WK3aFjSPD5UGEHVJ++gmEfa3ehi28oYlTiEVd\nN3K8Zj/CFSVtUKxzgyFpFKWdx/H0gCyxtIF0yQQggdWsdE4ZJe/LUjfjzHdTOzttIN67zqayeXrz\nXh04EOkLkuuAwXTPXYmfJR7TUb99/pl53XwpF7E6NfeON+WcazuZEgplKfjIQkSZVqJozvZf+VL7\n0LvVYoF4CY8Gf7Kdm/8nbgxc5dWY7olC9ou/ZB2bd8D8EH4upQhswKkIm6HycYGgAgVYoMRcQRQg\ntZVKVH0Oqg6em3Age+U2zlWmNwew7rNs2X15aOvOeKrh1K9H/CI9UMoL4p2YHa36mzzx9yZAzcb/\nLaEhILvbTHogX1NIMt9zC9SZ9pjDzw0RcA92Kr30A/YNWIZlzEv3V/QsTAJvZjzCOrXnNyo3UrO9\n+tYyicJMZ3zmjd3BIKOcufzycyuQuGTNHtGr8gc/xp1Wqy/2tSuuwTgbSo+PFP6nRQ5sjuiYySGV\neH9olKlifmV+ZYP/t82gbiezih2FDLFgWe9CU26R+O02j7+c5J+iKuC3j4mAeL/7qXmDYHZspCOk\nM8Mj71iPAYTtbd7NInp0F4ric7bvtI42bwT8zbwUzx1glHgBe75+tPlOCvtu8R6bpzeNf0RswzJF\n/VbYsfdk7lFhmmF7hAmIhMbYGnEAB8ZYjbi5CPZiViHVrwFZ29MsbAdBA2vAPkql2sZmH9deSbm/\nZNnYOmyjH6WvmiOgFlNXdFMouitAOE+cCeP1CM80o7Tn1ljoF1gWwlWC0ejx+UwW3UD7qfWkUEnk\nr3yOodSAAnT14tTpKnXUv8TU5GO7tMpGT7uElyNpR64LPNKxNELD/N4H+QpW6cGGPw/s74rNEEqw\nIm1NQ+jH1roQ/TvhVwRcmQ0LMjiyG09NFhj37vqI1QbXpABZJx//vLGreN9ItKgv+pXBzLJCJJA6\nT1ftr3KUoEcT8qNruW1Jee23XLjnv+KNwiTAYYJJlbH7pohpeRaIiZAPfyNhauO+FmlEOLWdKflN\nIyk9SQEKBcbCY5BPgqRT5//mw81D5U8mDebbZUABGiG4DBKGdub6fHkB+BeJnzV3UQ5OTNBxvW1W\nhhrzN6unv/ElNoqDqYhJwTEQT81e1a5aNvjpnIDguZZwFjBTdMyxXldbrMdE8+TByCBBApg5CWvo\n9HliP27Q9/jHe2T0g7bulg4qvGw7niJKDXwN+24o1TcGYN7B94dlDv1xVsPNQHQ92bgiLPjof8Nd\n/obqXZ7IslfiodUK+gCiuAgYtLZGAQUbgw7Kt0tuRqimpU1yZdOunvqdHfkEpC4IPFYM0TJpRZj/\nAQ3deoL+dPtNNTuqyC8UBTXoNAVFco2ifh9V1/C3/e2NkDGiy8933cngRVo9S9kkIGomauT9/pud\njIidngBwUJ/eeaY7uYUoIUdls0F1Fv1kjlLoxsMFqrbsQYhuzqsbIf/Z2LL/VsHDrCre1XQKbl9l\n617bynpyj3KlU/SxkjrdQfOGlVHfnsWwIdMCdy6Xxzr88LlDBSYSCrKnNKd4XZht4OKx8eO5wqDj\nMZbz8sfklD1IJ6YXoMIwAc3KQ/QZbQ2vg+NcG9IZoQTQSqkNhnlIcTNqoGWuhvKYUOXhijf/7VYq\nuFgMTYifcc9bZCq209abDbNP7AwHXUM9CsiHq+mmkS/kCqbERY47dBQUZTflFcPJPNvZf6ab/KvT\nQKbNKC7G9cRvq8dWgmmGGkh/vRl+w04psK/nIVhcvtLm1rzP9apTQ4u3qBbIGBLm7YCMkyAFw3Tf\ngJT71NztZbvccTLgL8ir1GnjnMDY2q9hGhgWWPlC+KyYwmgZ/2r16aCSX7Hwn3oRv+O+LyIN6pJo\nmdbpHZGIZ9sRbugCTUKJgi5ci19vKtfk6PGeuZeGP0tXZKxOxTOzEs2VTlqyHRQ+t0hQryek/US5\n+9UxZ9DVVPFArP85TlX91jOd8F+jSoT0JNdJNbDotUULXM

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAABAy21kYXQAAAKhBgX//53cRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTUyIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENv\ncHlsZWZ0IDIwMDMtMjAxNyAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9w\ndGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1o\nZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2\nIGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0\nX3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE2IGxvb2thaGVhZF90aHJlYWRz\nPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50ZXJsYWNlZD0wIGJsdXJheV9j\nb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBiX3B5cmFtaWQ9MiBiX2FkYXB0\nPTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29wPTAgd2VpZ2h0cD0yIGtleWlu\ndD0yNTAga2V5aW50X21pbj0yMCBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNoPTAgcmNfbG9va2Fo\nZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFwbWluPTAgcXBtYXg9\nNjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAADmJliIQAO//+906/AptFl2oD\nklcK9sqkJlm5UmsB8qYAAAMAAAMABMhfhdYZyUpNhAAAHeAB3jx+cIkfwCchP21cEfalXNXVgulS\nKskqpLIR+W5YUy258u3x+bOu98jkzQkzLsbwuLtWdkukVJ+080icqBR58ucBRiHw+O4d/qgKsuxR\n76t7vQ3NTUtb89iQl+/TQKydMlBrTrcouv9ojTvm+Gj9sPtSvOw9bXBFpq5Cz4AmFE6wHZ//4mJ9\ne2TxCloAjDARU5nbleBOawaEDKUeTrgpMA3ct1xnUNCx8Dz6lTEDLRaoiaAAAQA34gzmJ5or7LNH\nlTKXLHV8DAzoz0fzJxhxISbHkeU7gPQbMpzI6uFLuIFKhHFLhHNWC9eEjCBc1nWUbXC2nKDB77fT\nfx04q5l/Z90N96y7yZiRSsCmLnQhuhMM3Ecd9CZFFf+LDzVc16ilx9jI0Qh//I0LeYv9THqJGwKL\nnnEVOcRhqilX1qcmT6RWMjOXkhE9Fq3WDgarGR4Gm48kuQK8k6xz1YjvyPYgB0+MlXvI7jna2lQd\nkTcfoAAU227O/gBjU68xkJnQaO7BoUk4r2UJly94oAAAAwDUp97QYaQlNhQ/gVpDNBZvnTovtBDx\nw1b3nAJiFeF/PGbdA311r5ER5z3QHun1rG+1YeOQSdZUN6CD+lelGGzDPi6O73G036aGRiw8qoSQ\nySfMbxCFMuinO5CvuRDaNdp/zNCtHiNs3fGXzLkKWFpLfkf0NdHkBhkiawFyi989ClwZtW3BWw4w\nPmma07CbcEYJfhY/+Y3nViN6hvScqOI+raSqZQw3yooU2eOQzIYTMtJdsveb09bP/Xi/q5n/hQFB\nrvwpcMTr6LwDjDwiZqOMO9FaSmcvBmQYPnhQboNj/DrA0RjbvdRprFwXfUAAAKGljEfzVSrReMu9\npncGAfH6nzTWRLlaHnwwsunRP3+xPXtBBuDIeeNpMuz3P0O588M1rrVrLKIYhpp0q32NieLP39Qp\nNeWk1PwyR52ytsA4wuv6WjIWniu07ZzbW3yxDLTWGKH4CYSx5FgF4J+ZpmTxLZ7bsGyxx9a8FBuW\nj8fTtVcJryxdf9nOoRtZwPwd1SY3nYHTK0r77LtpZVKING8X0beK9/Q40VENKUItOp+lxa7i0DOL\n3B5z0A2919j/HxyOFcVR2SqO2NwYEp6EmbvGOL0dAFq3fr3E72Fy4bEOowG/Pk9AZZ6Eo+4edsUj\nJc4QqYZ/QUb22/vlZCdjw80Z6UnKNkj2ZbUS1GIZvNdbn04XiWnya2mFIokG2jEIIffXGAfa4Cp5\nEYsu+7OOKc55/ayjAymRPGjABN0Ua6++m5UzCvHxt31xyGAwFrEPylOFP6SDQWCgPw+E1ks/EZ9p\nRcs0fGr///R0uUiSXpSftq4NCa9rWSKEPw/IEl7+chG2v/Sev0MOscf5kkiAfF0gagKbQuem7aH9\nq9uADhXSqlU6C2OizN+W5GpV6k1MU+WK/WtzLFIRzq4VANOqhGDiRZd84/CerLSKbpKU4fIdH/t4\naIupY1Vn/H8eJGMhRsVqqKXe0qAkBovUKd1ya9WcavIgpQd+23HAl8lWgSvG0OE6RrgYTDWhlTOp\nHa9tGySRLjy+fGjt2WIozj9uIQxpdIkDXcKGPQZvl+c2Fm7RrkRAV1T2HL0SdYRAITcRPyKW+uPL\noq4aKTbtVhz6eqDz2arwuOePp62WuHRJJzR5x3GBpYKrfUrURQtHpGBRHvm773nxt/ZTE7ZOH9ZQ\nZBdlrVxtjVyOfMnvw5jhsoh8n/7JcloiiuZD5cYbq8nY4PBbCnDLl/ecj/WF/HU6hXEnMkBa6Qg3\n92VGW4lfG+Dn19GwUP4a1K3qTqBZ2SSOGa/7u1wl4dplMki+Mfyqc6shCq2LHoWats+SvaZqcdxE\nf/L9kRHVn/3JriV5h4dzwP8LGKEnButuwqbLSdYG0xEjqeIOBTYjHe3cuYZmeGE33CQqW8unLWvv\ndSfNgcmiLdzIDhYFVv6PiXjG2o+t4yzMA7LqhiJpx5/jtq54ID26aHD7EE+zNgpylQgHpNTdDT35\nnzLKVciZFyhKNuxbe50fn5ugFhFb034cQzUlrJLu88hwzaZV+JSKtWkukB2qLpy9wu/1rhizA4Jc\nbkhUcnS0w4Ro8S1UeT5wZlJITOIvPUE0a4VfbLJ4Xx1KCe3MIlKOfUdp22WasUkGNGqkC7OpsMHb\nlZhV8q9bdke04MXtWF1lp4qXNeMEty9j/nI0FwTxOnp3z/0oBzFP00RU2+C4XwrLdbYN2g/FVi+d\nhaXn5JhVPkIXb2noEy3ffiVh2SzydIRoxmlGExf2toEldfwjkvSz+JCZ6iCJ1vHBgGFVZzFa9k1n\nx/ZGAoOktZAXz8wMRDNOJqoUxgTxSfM/phAwABZ1VHSFJz9xSenjrfYptIKOhOD6/wLQyhZshPRv\niwIVP7yNd2F7+RxCzRns3VWJ1Hw0THEhh4ZC6caLX/RUbzFKElC0tOBSNEp1/UKvZ2vb3egyPXzd\nU2b9TkuZpgY/Olio+jxVnibIVP1ai0C+jCdfPkk8Hix7qOxExD7l/8ftj1cj9GUbcuZEj10EkzUU\naAaGXOoBHZCd09d/mLZlYwoL2eE16ZKkf91Zcg7AHN2Epgq/3M8uYdgKSyYCtN2E632tm/iL2uQ0\nnwAlNxwm861Bp55WxAczoDSfWsY6GnuldJrnz+N8WBAJeOa7XDa8Us8Xucq5kWrdYE7G0TIK2D//\n8v6vuJMJZxj0ZggAAAMAADOGQQvQEQtlvyKsa/obsvWb26

## MCMC

In [ ]:
@model three_body(ms, states, dt) = begin
    q ~ MvNormal(zeros(6), 10)
    p ~ MvNormal(zeros(6), 10)
    space = Space(Particle.(ms, q, p))
    for i in 1:length(states)
        space = transition(space, dt)
        states[i] ~ MvNormal(vec(stateof(space)), 1e-1)
    end
end

@time chn = sample(three_body(massof.(data.objs0), data.states, args.dt), args.alg, args.n_samples)

splot(chn; colordim=:parameter) |> display

bson(datadir("three_body-noise=$(args.σ).bson"), chn=chn)

;

let n_mc = 50, T = 100, do_mean = true, res = get(chn[end-n_mc+1:end], [:q, :p])
    Q = Matrix{Float64}(hcat(res.q...)')
    P = Matrix{Float64}(hcat(res.p...)')
    if do_mean
        Q = mean(Q; dims=2)
        P = mean(P; dims=2)
    end
    Q̂ = zeros(6, T)
    for i in 1:size(Q, 2)
        q, p = Q[:,i], P[:,i]
        space = Space(Particle.(data.ms, q, p))
        traj = simulate(space, args.dt, T)
        Q̂ = Q̂ + hcat(positionof.(traj)...)
    end
    Q̂ = Q̂ / size(Q, 2)
    HTML(animof(Q̂).to_html5_video()) |> display
end

;